In [ ]:
%%capture
!pip install datasets
# ⚠️ Reinicia el runtime después de correr esto la primera vez
!pip uninstall -y torch triton xformers unsloth transformers
!pip install torch==2.6.0+cu124 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers==4.51.3
!pip install triton==3.2.0
!pip install xformers==0.0.29.post3
#!pip install unsloth==2025.4.1


In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Crear Dataset de wordlist
"""
import pandas as pd
import gzip
import random

# Ruta base
base_path = '/content/drive/MyDrive/Train Family/'

# Familias basadas en listas de palabras
wordlist_families = ['suppobox', 'charbot', 'gozi', 'matsnu', 'manuelita', 'rovnix', 'deception', 'nymaim']

# Inicializamos una lista para guardar los DataFrames
dfs = []

# Para cada familia
for family in wordlist_families:
    file_path = base_path + family + '.gz'
    with gzip.open(file_path, 'rt') as f:
        df = pd.read_csv(f)

    # Nos quedamos solo con las columnas deseadas
    df = df[['domain', 'family', 'label']]

    # Muestreamos 18000 dominios aleatoriamente
    df_sampled = df.sample(n=10000, random_state=42)

    dfs.append(df_sampled)

# Ahora cargamos los dominios normales (legit.gz)
file_path_legit = base_path + 'legit.gz'
with gzip.open(file_path_legit, 'rt') as f:
    df_legit = pd.read_csv(f)

df_legit = df_legit[['domain', 'family', 'label']]

# Muestreamos 18000 normales
df_legit_sampled = df_legit.sample(n=80000, random_state=42)

dfs.append(df_legit_sampled)

# Unimos todo
train_wl = pd.concat(dfs).reset_index(drop=True)

# Verificamos
print(train_wl['family'].value_counts())
print(train_wl.head())

# Si querés guardar el dataset resultante:
train_wl.to_csv('/content/drive/MyDrive/MOE_DGA/train_wl.csv', index=False)
"""

"\nimport pandas as pd\nimport gzip\nimport random\n\n# Ruta base\nbase_path = '/content/drive/MyDrive/Train Family/'\n\n# Familias basadas en listas de palabras\nwordlist_families = ['suppobox', 'charbot', 'gozi', 'matsnu', 'manuelita', 'rovnix', 'deception', 'nymaim']\n\n# Inicializamos una lista para guardar los DataFrames\ndfs = []\n\n# Para cada familia\nfor family in wordlist_families:\n    file_path = base_path + family + '.gz'\n    with gzip.open(file_path, 'rt') as f:\n        df = pd.read_csv(f)\n\n    # Nos quedamos solo con las columnas deseadas\n    df = df[['domain', 'family', 'label']]\n\n    # Muestreamos 18000 dominios aleatoriamente\n    df_sampled = df.sample(n=10000, random_state=42)\n\n    dfs.append(df_sampled)\n\n# Ahora cargamos los dominios normales (legit.gz)\nfile_path_legit = base_path + 'legit.gz'\nwith gzip.open(file_path_legit, 'rt') as f:\n    df_legit = pd.read_csv(f)\n\ndf_legit = df_legit[['domain', 'family', 'label']]\n\n# Muestreamos 18000 normales\

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.2: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
import pandas as pd

# Carga tu dataset desde CSV
file_path = '/content/drive/My Drive/MOE_DGA/train_wl.csv'
df = pd.read_csv(file_path)




# Asumiendo que ya tienes el DataFrame `df`

# Separar dominios legítimos y dga
df_legit = df[df['family'] == 'legit']
df_dga = df[df['label'] == 'dga']

# Tomar una muestra de 40000 dominios legítimos
df_legit_sample = df_legit.sample(n=40000, random_state=42)

# Tomar 5000 dominios por familia DGA
df_dga_sample = (
    df_dga.groupby('family', group_keys=False)
    .apply(lambda x: x.sample(n=5000, random_state=42))
)

# Concatenar los resultados
df_final = pd.concat([df_legit_sample, df_dga_sample], ignore_index=True)
df = df_final
# Verificar
print(df['family'].value_counts())



family
legit        40000
charbot       5000
deception     5000
gozi          5000
manuelita     5000
matsnu        5000
nymaim        5000
rovnix        5000
suppobox      5000
Name: count, dtype: int64


<ipython-input-10-29c233f01233>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=5000, random_state=42))


In [ ]:
import pandas as pd
from datasets import Dataset
import json




# Crear conversaciones a partir de input_text y Label
conversations = []
for _, row in df.iterrows():
    domain = row['domain']
    label = row['label']

    input_text = f"Classify the following domain name as dga or normal: {domain}"


    # Crear respuesta ficticia basada en Label
    if label == "dga":
        response = "The domain name is classified as dga."
    else:
        response = "The domain name is classified as normal."

    # Crear estructura de conversación
    conversation = {
        "conversations": [
            {"from": "human", "value": input_text},
            {"from": "gpt", "value": response}
        ]
    }
    conversations.append(conversation)

# Convertir a Hugging Face Dataset
dataset = Dataset.from_list(conversations)

In [ ]:
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import Dataset

# Preparar el tokenizador con la plantilla de chat
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# Función para formatear prompts
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Estandarizar el dataset (ShareGPT)
dataset = standardize_sharegpt(dataset)

# Mapear el formato para aplicar la plantilla del tokenizador
dataset = dataset.map(formatting_prompts_func, batched=True)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
dataset[0]

{'conversations': [{'content': 'Classify the following domain name as dga or normal: bw87wfohbwuvbg.com',
   'role': 'user'},
  {'content': 'The domain name is classified as normal.',
   'role': 'assistant'}],
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nClassify the following domain name as dga or normal: bw87wfohbwuvbg.com<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe domain name is classified as normal.<|eot_id|>'}

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 128,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 80,000 | Num Epochs = 1 | Total steps = 156
O^O/ \_/ \    Batch size per device = 128 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (128 x 4 x 1) = 512
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.561400
2,1.546400
3,1.408600
4,0.901100
5,0.424700
6,0.150100
7,0.093900
8,0.073600
9,0.063100
10,0.073200


In [ ]:
# Guardar el modelo y el tokenizador
trainer.model.save_pretrained("outputs/model")
tokenizer.save_pretrained("outputs/model")


('outputs/model/tokenizer_config.json',
 'outputs/model/special_tokens_map.json',
 'outputs/model/tokenizer.json')

In [ ]:
# Definimos la ruta donde queremos guardar
save_directory = "/content/drive/MyDrive/MOE_DGA/outputs/model"

# Guardamos el modelo y el tokenizador
trainer.model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('/content/drive/MyDrive/MOE_DGA/outputs/model/tokenizer_config.json',
 '/content/drive/MyDrive/MOE_DGA/outputs/model/special_tokens_map.json',
 '/content/drive/MyDrive/MOE_DGA/outputs/model/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Classify the following domain name as dga or normal: visitform.net"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.7, min_p = 0.1)

The domain name is classified as dga.<|eot_id|>


In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import sys
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import re

families = [
    #'matsnu.gz'#,
    #'suppobox.gz',
    #'charbot.gz',
    #'gozi.gz',
    'manuelita.gz'#,
    #'rovnix.gz',
    #'deception.gz',
    #'nymaim.gz'
]

runs = 30

for family in families:
    print(family)
    dga = pd.read_csv(f'/content/drive/My Drive/Familias_Test/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)
    dfs = []
    for run in range(runs):
        print(f'{run:2}/{runs}', end='\r')
        dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])
        pred = []
        prob = []
        query_time = []
        results = []

        for domain_to_check in dfw.domain.values:
            st = time.time()
            messages = [{"role": "user", "content": "Classify the following domain name as dga or normal:" + domain_to_check}]
            inputs = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
            ).to("cuda")

            # Generar texto completo sin streaming (más fácil para post-procesar)
            output_ids = model.generate(
                input_ids=inputs,
                max_new_tokens=128,
                use_cache=True,
                temperature=0.7,
                do_sample=False,
                top_p=1.0,
            )
            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            # Ejemplo de salida: "The traffic is classified as normal."
            # Parseamos la respuesta:
            if "classified as dga" in generated_text.lower():
                label_value = 1
            elif "classified as normal" in generated_text.lower():
                label_value = 0
            else:
                # Caso no esperado, asignar 0 o manejar de otra forma
                label_value = 0

            pred.append(label_value)
            query_time.append(time.time() - st)

        dfw['pred'] = pred
        # dfw['prob'] = prob  # Si tienes probabilidades, descomenta esta línea
        dfw['query_time'] = query_time
        dfw.to_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz', index=False)


manuelita.gz


In [ ]:
"""
families = [
    'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz',
    'manuelita.gz',
    'rovnix.gz',
    'deception.gz',
    'nymaim.gz'
]
"""

def fpr_tpr(y, ypred):
    tn, fp, fn, tp = confusion_matrix(y, ypred).ravel()
    fpr = fp / (fp + tn)  # False Positive Rate
    tpr = tp / (tp + fn)  # True Positive Rate (Recall)
    return fpr, tpr

for family in families:
    acc = []
    pre = []
    rec = []
    f1 = []
    fpr = []
    tpr = []
    qt = []
    qts = []
    for run in range(runs):
        df = pd.read_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz')
        y = (df.label == 'dga').astype(int)
        ypred = df.pred
        acc.append(accuracy_score(y, ypred))
        pre.append(precision_score(y, ypred))
        rec.append(recall_score(y, ypred))
        f1.append(f1_score(y, ypred))
        fpr_value, tpr_value = fpr_tpr(y, ypred)
        fpr.append(fpr_value)
        tpr.append(tpr_value)
        qt.append(df.query_time.mean())
        qts.append(df.query_time.std())
#    print(f'Query time: {np.mean(qt):0.5f}+/-{np.mean(qts)}:0.5f')
    print(f'{family.split(".")[0]:15}: acc:{np.mean(acc):0.2f}±{np.std(acc):.3f} f1:{np.mean(f1):0.2f}±{np.std(f1):.3f} pre:{np.mean(pre):0.2f}±{np.std(pre):.3f} rec:{np.mean(rec):0.2f}±{np.std(rec):.3f}  FPR:{np.mean(fpr):0.2f}±{np.std(fpr):.3f} TPR:{np.mean(tpr):0.2f}±{np.std(tpr):.3f} Query time: {np.mean(qt):0.5f}±{np.mean(qts):0.5f}')


manuelita      : acc:0.64±0.038 f1:0.50±0.072 pre:0.80±0.081 rec:0.37±0.068  FPR:0.09±0.046 TPR:0.37±0.068 Query time: 0.59484±0.09698
